In [1]:
import tweepy


In [2]:
# Save your consumer key, consumer secret, access token, and access secret here.
# Don't share these secrets with others!  If you're building code for Twitter which many people will share,
# you should encrypt this information.  
# It's also possible to generate access tokens and secrets from within an app.

con_key = input('Enter consumer key:')
con_secret = input ('Enter consumer secrect:')
acc_token = input('Enter Access Token:')
acc_secret = input('Enter access secret:')

In [3]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) 

In [6]:
#Download tweets from Twitter based upon searchQuery criteria
import sys
import jsonpickle
import os

searchQuery = '#%23elevateyourstyle+OR+#%23NewInDL+OR+#%23StitchFix+OR+#%23TrunkClub+OR+#%23Wantable+OR+##%23stelladot+OR+#%23stelladotstyle'#this is what we're searching for
maxTweets = 1000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = input('Enter a filename: ') #Prompt user to enter file name each time it is run

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                    print (searchQuery)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

Enter a filename: tweets_test.txt
#%23elevateyourstyle+OR+#%23NewInDL+OR+#%23StitchFix+OR+#%23TrunkClub+OR+#%23Wantable+OR+##%23stelladot+OR+#%23stelladotstyle
Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1000 tweets, Saved to tweets_test.txt


In [8]:
import json
from pprint import pprint

In [10]:
#Make user interactive if time
#Import Json File  
first_file=input('Is this the first file Yes or No:')
fileName=input('Enter a filename: ')
#fileName='tweets_0805am.txt'

#first_file= 'No' #Can change this variable upon first file load.  Saves multiple files to a single list

#Reinitialize file for loading into list using variable first_file.  Resets to empty if first_file = 'Yes'
if first_file == 'Yes': 
    tweets = []
else:
    tweets=tweets
    
for line in open(fileName, 'r'):
    tweets.append(json.loads(line))


Is this the first file Yes or No:Yes
Enter a filename: tweets_test.txt


In [11]:
#Check list length.  Confirm if able to append multiple files
len(tweets)

1000

In [12]:
#Create list of items interested in analyzing 
twitter_list = [(item['created_at'], item['id'], item['id_str'], item['entities'].get('hashtags'), len(item['entities']['hashtags']), 
    item['favorited'], item ['favorite_count'], item['retweeted'], 
    item['retweet_count'], item['text'], item['user'].get('name'), item['user'].get('screen_name'), item['user'].get('id'), item['user'].get('id_str'),
     item['user'].get('friends_count')) for item in tweets]

In [13]:
#Transform list into dataframe
import pandas as pd
twitter_df = pd.DataFrame(twitter_list)
column_names = ("created_at", "id", "id_string", "hashtags", "hashtag_count", "favorited", "favorite_count", 
                "retweeted", "retweet_count", "tweet_text", "user_name", "user_screen_name", "user_id", "user_id_string", "user_friends_count")
twitter_df.columns=column_names
len(twitter_df)


1000

In [14]:
#Access text value syntax
twitter_df['hashtags'][0][0]['text']

'justkeepsgettingbetterandbetter'

In [15]:
#Iterate through nest dicts to get hashtag text
ht=[]
for s in range(len(twitter_df['hashtags'])):
    hasht=[]
    for t in range(len(twitter_df.hashtags[s])):
        hasht.append(twitter_df['hashtags'][s][t]['text'])
        t+t+1
    ht.append(hasht)
    s=s+1
 

In [16]:
#Create series and add column to dataframe with list of text values
htse=pd.Series(ht)
twitter_df['hashtag_list']=htse.values


In [17]:
#transform list into string
twitter_df['hashtag_list'] = twitter_df.hashtag_list.apply(', '.join)


In [18]:
#create dataframe column containing stitchfix boolean indicator
twitter_df['stitchfix_boolean']=twitter_df['hashtag_list'].str.lower().str.contains('stitchfix')

In [19]:
#create dataframe column containing stelladot boolean indicator
twitter_df['stelladot_boolean']=twitter_df['hashtag_list'].str.lower().str.contains('stelladotstyle')

In [20]:
#Review dataframe values
twitter_df.describe()

,id,hashtag_count,favorited,favorite_count,retweeted,retweet_count,user_id,user_friends_count,stitchfix_boolean,stelladot_boolean
count,1.000000e+03,1000.000000,1000,1000.000000,1000,1000.000000,1.000000e+03,1000.000000,1000,1000
mean,8.929401e+17,1.548000,0,0.141000,0,0.481000,5.568270e+17,2315.518000,0.138,0.765
std,6.390853e+14,1.202973,0,0.693974,0,2.730768,3.178214e+17,2551.803484,0.345073,0.424211
min,8.918581e+17,0.000000,False,0.000000,False,0.000000,7.268392e+06,0.000000,False,False
25%,8.923958e+17,1.000000,0,0.000000,0,0.000000,7.030516e+17,390.000000,0,1
50%,8.929314e+17,1.000000,0,0.000000,0,0.000000,7.057811e+17,643.000000,0,1
75%,8.935141e+17,1.000000,0,0.000000,0,0.000000,7.656281e+17,4728.000000,0,1
max,8.940323e+17,10.000000,False,16.000000,False,20.000000,8.840536e+17,31729.000000,True,True


In [21]:
#Check data types of fields before exporting
twitter_df.dtypes

created_at            object
id                     int64
id_string             object
hashtags              object
hashtag_count          int64
favorited               bool
favorite_count         int64
retweeted               bool
retweet_count          int64
tweet_text            object
user_name             object
user_screen_name      object
user_id                int64
user_id_string        object
user_friends_count     int64
hashtag_list          object
stitchfix_boolean       bool
stelladot_boolean       bool
dtype: object

In [22]:
#sort dataframe by ID and user looking for duplicates
twitter_df.sort_values(by=['id', 'retweet_count', 'favorite_count', 'user_friends_count'], axis=0, inplace=True)
len(twitter_df)

1000

In [23]:
#Drop duplicates identified by id column keeping first column getting latest update on tweet
twitter_dfdedup=twitter_df.drop_duplicates(subset=['id'], keep='last')
len(twitter_dfdedup)

1000

In [24]:
#Create output file for analysis in R
fileNameOut=input('Enter an output filename: ')
twitter_dfdedup.to_csv(fileNameOut, columns=["created_at", "id", "id_string", "hashtags", "hashtag_count", "favorited", 
                "favorite_count", "retweeted", "retweet_count", "user_name","user_screen_name", 
                "user_id", "user_id_string", "user_friends_count", "hashtag_list", "stitchfix_boolean", "stelladot_boolean" ], na_rep="", index=False)

Enter an output filename: pandas_out.csv
